In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
excel = pd.ExcelFile('HainanData_kindofClean.xlsx')
hainan = excel.parse("Clean")
hainan.columns = hainan.columns.str.replace('\s+', '_')
hainan.columns = hainan.columns.str.replace('(', '')
hainan.columns = hainan.columns.str.replace(')', '')
hainan.drop(['Day', 'Year', 'Water/m3', 'Total_electricity_cons_kWh', 
             '50%_NaOH/kg', 'FeCl2/kg', 'PAM/kg', 'Defoamer/kg', 'day_#2'], axis = 1, inplace = True)
d = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6,
     'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
hainan.Month = hainan.Month.map(d)
hainan.BioCNG_Produced_Nm3 = hainan.BioCNG_Produced_Nm3.shift(-15)
hainan.drop(hainan.tail(15).index,inplace=True)
hainan['BioCNG_cumsum'] = hainan.BioCNG_Produced_Nm3.cumsum()
print(hainan.columns)

In [ ]:
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
print('train data len:',len(hainan_train))
print('test data len:',len(hainan_test))
hainan
hainan = hainan[np.isfinite(hainan['Month'])]
print(hainan.isnull().sum())

In [ ]:
#ols
hainan_ols = ols("BioCNG_cumsum ~ Manure_input_t + Bagasse_1_input_t + Lees_fermentation +\
                    Fish_waste_input_t + Alcowaste_input_t + Cassava_input_t + Grass_input_t +\
                    Tea_waste_input_t + Other_input_t ", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary

In [5]:
#ols
hainan_ols = ols("Liquid_Fertilizer_Produced_t ~ Manure_input_t + Bagasse_1_input_t + Lees_fermentation +\
                    Fish_waste_input_t + Alcowaste_input_t + Cassava_input_t + Grass_input_t +\
                    Tea_waste_input_t + Other_input_t ", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     Liquid_Fertilizer_Produced_t   R-squared:                       0.080
Model:                                      OLS   Adj. R-squared:                  0.070
Method:                           Least Squares   F-statistic:                     7.802
Date:                          Fri, 02 Mar 2018   Prob (F-statistic):           4.45e-11
Time:                                  17:18:01   Log-Likelihood:                -4408.2
No. Observations:                           816   AIC:                             8836.
Df Residuals:                               806   BIC:                             8883.
Df Model:                                     9                                         
Covariance Type:                      nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             19.4213      3.606      5.385      0.000      12.343      26.500
Manure_input_t         0.8652      0.231      3.746      0.000       0.412       1.319
Bagasse_1_input_t      0.2030      0.128      1.589      0.112      -0.048       0.454
Lees_fermentation      0.3230      0.264      1.225      0.221      -0.195       0.841
Fish_waste_input_t     1.2228      0.328      3.725      0.000       0.579       1.867
Alcowaste_input_t      0.4469      0.436      1.025      0.305      -0.408       1.302
Cassava_input_t        0.3431      0.070      4.900      0.000       0.206       0.481
Grass_input_t         -0.2185      0.249     -0.876      0.381      -0.708       0.271
Tea_waste_input_t      0.2827      1.012      0.279      0.780      -1.703       2.269
Other_input_t          0.0481      0.060      0.799      0.425      -0.070       0.166
==============================================================================
Omnibus:                      228.341   Durbin-Watson:                   1.933
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              509.214
Skew:                           1.528   Prob(JB):                    2.66e-111
Kurtosis:                       5.373   Cond. No.                         98.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
#ols
hainan_ols = ols("Solid_fertilizer_produced_t ~ Manure_input_t + Bagasse_1_input_t + Lees_fermentation +\
                    Fish_waste_input_t + Alcowaste_input_t + Cassava_input_t + Grass_input_t +\
                    Tea_waste_input_t + Other_input_t ", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Solid_fertilizer_produced_t   R-squared:                       0.069
Model:                                     OLS   Adj. R-squared:                  0.059
Method:                          Least Squares   F-statistic:                     6.669
Date:                         Fri, 02 Mar 2018   Prob (F-statistic):           3.06e-09
Time:                                 17:18:06   Log-Likelihood:                -3256.6
No. Observations:                          816   AIC:                             6533.
Df Residuals:                              806   BIC:                             6580.
Df Model:                                    9                                         
Covariance Type:                     nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              1.1935      0.879      1.357      0.175      -0.533       2.920
Manure_input_t        -0.0037      0.056     -0.065      0.948      -0.114       0.107
Bagasse_1_input_t      0.1481      0.031      4.753      0.000       0.087       0.209
Lees_fermentation      0.1414      0.064      2.200      0.028       0.015       0.268
Fish_waste_input_t     0.2272      0.080      2.839      0.005       0.070       0.384
Alcowaste_input_t     -0.1410      0.106     -1.327      0.185      -0.350       0.068
Cassava_input_t        0.0305      0.017      1.789      0.074      -0.003       0.064
Grass_input_t         -0.0156      0.061     -0.256      0.798      -0.135       0.104
Tea_waste_input_t      0.1629      0.247      0.660      0.509      -0.321       0.647
Other_input_t          0.0567      0.015      3.858      0.000       0.028       0.086
==============================================================================
Omnibus:                      691.254   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14827.134
Skew:                           3.860   Prob(JB):                         0.00
Kurtosis:                      22.403   Cond. No.                         98.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

From the ols results based on three different dependent variable, we decided to focus on the first one.

In [8]:
# Predict on the training data
X_train = hainan_train[['Manure_input_t', 'Bagasse_1_input_t',
       'Lees_fermentation', 'Fish_waste_input_t', 'Alcowaste_input_t',
       'Cassava_input_t', 'Grass_input_t', 'Tea_waste_input_t',
       'Other_input_t']]
y_train = hainan_train.BioCNG_Produced_Nm3
y_pred_train = hainan_ols.predict(X_train)

# Compute the root-mean-square of training data
rms_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print('Training error:',rms_train/sum(y_train))

# Predict on the test data
X_test = hainan_test[['Manure_input_t', 'Bagasse_1_input_t',
       'Lees_fermentation', 'Fish_waste_input_t', 'Alcowaste_input_t',
       'Cassava_input_t', 'Grass_input_t', 'Tea_waste_input_t',
       'Other_input_t']]
y_test = hainan_test.BioCNG_Produced_Nm3
y_pred_test = hainan_ols.predict(X_test)

# Compute the root-mean-square of test data
rms_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print('Testing error:',rms_test/sum(y_test))

Training error: 0.00147106480219
Testing error: 0.00577617250521


In [9]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of logistic regression classifier on train set: 0.17
Accuracy of logistic regression classifier on test set: 0.01
[[3 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 1 0 0]]


In [10]:
# SVM
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)  
y_pred_test = clf.predict(X_test)
print('Accuracy of SVM on train set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of SVM on test set: {:.2f}'.format(clf.score(X_test, y_test)))

Accuracy of SVM on train set: 0.70
Accuracy of SVM on test set: 0.01


In [11]:
# perceptron
from sklearn.linear_model import perceptron
net = perceptron.Perceptron(n_iter=100, verbose=0, random_state=None, fit_intercept=True, eta0=0.002)
net.fit(X_train, y_train)
y_pred_test = net.predict(X_test)
print('Accuracy of perceptron on train set: {:.2f}'.format(net.score(X_train, y_train)))
print('Accuracy of perceptron on test set: {:.2f}'.format(net.score(X_test, y_test)))

/Users/anna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Accuracy of perceptron on train set: 0.02
Accuracy of perceptron on test set: 0.00


In [12]:
# kNN
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(hainan)
hainan_normalized = pd.DataFrame(np_scaled)
hainan_normalized_train, hainan_normalized_test = train_test_split(hainan_normalized, test_size=0.2)
print(hainan_normalized_train.head())

Xnor_train = hainan_normalized_train[[10,11,12,13,14,15,16,17,18]]
ynor_train = hainan_normalized_train[21]
Xnor_test = hainan_normalized_test[[10,11,12,13,14,15,16,17,18]]
ynor_test = hainan_normalized_test[21]
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=4)
knn.fit(Xnor_train, ynor_train)
y_pred_test = knn.predict(Xnor_test)
print('Accuracy of kNN on train set: {:.2f}'.format(knn.score(Xnor_train, ynor_train)))
print('Accuracy of kNN on test set: {:.2f}'.format(knn.score(Xnor_test, ynor_test)))

           0         1         2         3         4         5         6   \
230  1.000000  0.212121  0.225711  0.000000  0.343670  0.251059  0.187443   
774  0.454545  0.757576  0.761531  0.158389  0.303670  0.000000  0.000000   
113  0.636364  0.090909  0.110893  0.000000  0.187566  0.231579  0.000000   
999  0.000000  0.969697  0.982336  0.480884  0.696404  0.472051  0.701773   
753  0.363636  0.727273  0.738960  0.209916  0.188165  0.000000  0.000000   

           7         8         9     ...      12        13       14   15  \
230  0.187443  0.075319  0.075319    ...     0.0  0.000000  0.00000  0.0   
774  0.000000  0.502068  0.502068    ...     0.0  0.288945  0.00000  0.0   
113  0.000000  0.000000  0.000000    ...     0.0  0.000000  0.00000  0.0   
999  0.701773  0.000000  0.000000    ...     0.0  0.000000  0.14898  0.0   
753  0.000000  0.497242  0.497242    ...     0.0  0.000000  0.00000  0.0   

           16   17        18        19        20        21  
230  0.000000  0.0 

In [13]:
# random forest
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators = 1000)
random_forest.fit(X_train, y_train)
print('Accuracy of Random Forest on train set: {:.2f}'.format(random_forest.score(X_train, y_train)))
print('Accuracy of Random Forest on test set: {:.2f}'.format(random_forest.score(X_test, y_test)))

Accuracy of Random Forest on train set: 0.86
Accuracy of Random Forest on test set: 0.45


In [ ]:
# xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred_test = xgb.predict(X_test)
print('Accuracy of XGboost on train set: {:.2f}'.format(xgb.score(X_train, y_train)))
print('Accuracy of XGboost on test set: {:.2f}'.format(xgb.score(X_test, y_test)))